In [2]:
from scipy import sparse
from scipy.sparse import linalg
import functools

$$\sigma_z = \begin{pmatrix} 1 & 0 \\ 0 & -1\end{pmatrix}.$$

In [3]:
sigma_z = sparse.bsr_matrix([[1, 0], [0, -1]])

`sigma_product` returns the direct product of $n$ $\sigma_z$'s such that $2^n = d$, i.e.
$$\sigma_z \otimes \sigma_z \otimes \cdots \otimes \sigma_z = \begin{pmatrix} \ddots \end{pmatrix}_{d\times d}.$$

In [4]:
def sigma_product(dimension):
    dimension = int(dimension)
    return (
        sparse.bsr_matrix([[1]])
    ) if dimension == 1 else (
        sparse.kron(sigma_z, sigma_product(dimension / 2))
    )

Parameters: $t$ and $U$ in the Hamiltonian, as well as the length of the chain $L$. Our Hamiltonian is given by
$$H = -t \sum_\sigma \sum_{i=0}^{L-2} c^\dagger_{i,\sigma} c_{i+1,\sigma} + U \sum_{i=0}^{L-1} c^\dagger_{i,\uparrow}c_{i,\uparrow}c^\dagger_{i,\downarrow}c_{i,\downarrow}.$$

In [5]:
t = 0.5 # hopping t
U = 1.0 # on-site U
L = 4   # number of sites

The creation and operator of each spin on a single site is given by
$$\begin{gather*}
c_\uparrow = \begin{pmatrix}0 & 0 \\ 1 & 0\end{pmatrix} \otimes \begin{pmatrix}1 & 0 \\ 0 & 1\end{pmatrix},\\
c_\downarrow = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix} \otimes \begin{pmatrix}0 & 0 \\ 1 & 0\end{pmatrix}.
\end{gather*}$$

In [6]:
d_spin          = 2 # number of states on each site of a fixed spin: occupied / not occupied
d_site          = d_spin * d_spin # number of states on each site
creation        = sparse.bsr_matrix([[0, 0], [1, 0]], dtype='d') # occupied as [0, 1]' and not occupied as [1, 0]'
up_creation     = sparse.kron(creation, sparse.eye(d_spin))
down_creation   = sparse.kron(sigma_product(d_spin), creation) # np.kron(util.sigma_product(model_d), creation)

Now we calculate the creation operators of each spin in the Hilbert space of the whole system, i.e. matrices of size $4^L \times 4^L$. They are given by
$$\begin{gather*}
c_{i,\uparrow} = \underbrace{\sigma_z \otimes \sigma_z \otimes \cdot \otimes \sigma_z}_{2i} \otimes c_{\uparrow} \otimes \underbrace{\begin{pmatrix}0 & 0 \\ 1 & 0\end{pmatrix} \otimes \begin{pmatrix}0 & 0 \\ 1 & 0\end{pmatrix} \otimes \cdots \otimes \begin{pmatrix}0 & 0 \\ 1 & 0\end{pmatrix}}_{2(L - 1 - i)}, \\
c_{i,\downarrow} = \underbrace{\sigma_z \otimes \sigma_z \otimes \cdot \otimes \sigma_z}_{2i} \otimes c_{\downarrow} \otimes \underbrace{\begin{pmatrix}0 & 0 \\ 1 & 0\end{pmatrix} \otimes \begin{pmatrix}0 & 0 \\ 1 & 0\end{pmatrix} \otimes \cdots \otimes \begin{pmatrix}0 & 0 \\ 1 & 0\end{pmatrix}}_{2(L - 1 - i)}.
\end{gather*}$$

In [7]:
up_creation_operators, down_creation_operators = ([
        functools.reduce(sparse.kron,
            (sigma_product(d_site ** i), creation, sparse.eye(d_site ** (L - 1 - i)))
        )
        for i in range(L)
    ] for creation in (up_creation, down_creation)
)

Now we calculate the matrix representations of the hopping terms, i.e.
$$-t \begin{pmatrix} c^\dagger_{0,\uparrow} c_{1,\uparrow} + \mathrm{h.c.} & c^\dagger_{1,\uparrow} c_{2,\uparrow} + \mathrm{h.c.} & \cdots & c^\dagger_{L-2,\uparrow} c_{L-1,\uparrow} + \mathrm{h.c.} \end{pmatrix}$$
and
$$-t \begin{pmatrix} c^\dagger_{0,\downarrow} c_{1,\downarrow} + \mathrm{h.c.} & c^\dagger_{1,\downarrow} c_{2,\downarrow} + \mathrm{h.c.} &  \cdots & c^\dagger_{L-2,\downarrow} c_{L-1,\downarrow} + \mathrm{h.c.} \end{pmatrix}.$$

In [8]:
up_hoppings, down_hoppings = (map(
        lambda creation_0, creation_1: (
            -t * ((creation_0 * creation_1.H) + (creation_0 * creation_1.H).H)
        ),
        operators[:-1], operators[1:]
    ) for operators in (up_creation_operators, down_creation_operators)
)

Then we calculate the matrix representations of the on-site terms, i.e.
$$U \begin{pmatrix} c^\dagger_{0,\uparrow} c_{0,\uparrow} c^\dagger_{0,\downarrow} c_{0,\downarrow} & c^\dagger_{1,\uparrow} c_{1,\uparrow} c^\dagger_{1,\downarrow} c_{1,\downarrow} & \cdots & c^\dagger_{L-1,\uparrow} c_{L-1,\uparrow} c^\dagger_{L-1,\downarrow} c_{L-1,\downarrow} \end{pmatrix}.$$

In [9]:
on_sites = map(
    lambda creation_up, creation_down: (
        U * creation_up * creation_up.H * creation_down * creation_down.H
    ),
    up_creation_operators, down_creation_operators
)

Now we sum the terms above to get the matrix representation of the Hamiltonian.

In [10]:
H = sum(up_hoppings) + sum(down_hoppings) + sum(on_sites)

assert H.shape == (d_site ** L, d_site ** L)

Finally we diagonalize the Hamiltonian and obtain the minimal eigenvalue, which is then devided by $L$ to get the energy per site.

In [13]:
min_E = linalg.eigsh(H, k=1, which="SA")[0][0]

print(f"Energy per site: {min_E / L}.")

Energy per site: -0.38369191991780155.
